# 

# OmniBERT model for hate speech detection/text classification

### Input Text Below

In [1]:
input_text = "value"

## Code for Text Classification with BERT from Huggingface
Using Transformers, Datasets, Evaluate and Accelerate from Huggingface

In [1]:
from huggingface_hub import notebook_login

notebook_login()

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'acl.id', 'X1', 'text', 'label', 'type', 'target', 'level', 'split', 'round.base', 'annotator', 'round', 'acl.id.matched'],
        num_rows: 82399
    })
})


Found cached dataset csv (/home/datalore/.cache/huggingface/datasets/LennardZuendorf___csv/LennardZuendorf--Dynamically-Generated-Hate-Speech-Dataset-efbf30a9d2cd70f9/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


In [ ]:
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tonkenized_data = dataset.map(preprocess_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
import evaluate
import numpy as np

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

id2label = {0: "nothate", 1: "hate"}
label2id = {"nothate": 0, "hate": 1}

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model="stevhliu/my_awesome_model")
classifier(input_text)